In [128]:
import pandas as pd

In [129]:
foundation_folder = "FoodData_Central_foundation_food_csv_2023-10-26"

foundation_food_df = pd.read_csv(f'data_source/{foundation_folder}/food.csv')
foundation_food_nutrient_df = pd.read_csv(f'data_source/{foundation_folder}/food_nutrient.csv')
foundation_nutrient_df = pd.read_csv(f'data_source/{foundation_folder}/nutrient.csv')

C:\Users\Michael\AppData\Local\Temp\ipykernel_26552\2253104221.py:4: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  foundation_food_nutrient_df = pd.read_csv(f'data_source/{foundation_folder}/food_nutrient.csv')


In [130]:
foundation_food_df.info()
# remove description is null data
foundation_food_df = foundation_food_df[foundation_food_df.description.notna()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60432 entries, 0 to 60431
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fdc_id            60432 non-null  int64  
 1   data_type         60432 non-null  object 
 2   description       60424 non-null  object 
 3   food_category_id  58732 non-null  float64
 4   publication_date  60432 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.3+ MB


In [131]:
foundation_food_nutrient_df.info()
# remove amount is null data
foundation_food_nutrient_df = foundation_food_nutrient_df[foundation_food_nutrient_df.amount.notna()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140824 entries, 0 to 140823
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 140824 non-null  int64  
 1   fdc_id             140824 non-null  int64  
 2   nutrient_id        140824 non-null  int64  
 3   amount             140804 non-null  float64
 4   data_points        137625 non-null  float64
 5   derivation_id      140681 non-null  float64
 6   min                13825 non-null   float64
 7   max                13825 non-null   float64
 8   median             14427 non-null   float64
 9   footnote           3 non-null       object 
 10  min_year_acquired  28689 non-null   object 
dtypes: float64(6), int64(3), object(2)
memory usage: 11.8+ MB


In [132]:
foundation_nutrient_df.info()
# only keep the target nutrition I want to track
tracking_nutrition = {"Protein", "Total lipid (fat)", "Carbohydrate, by difference"}
foundation_nutrient_df = foundation_nutrient_df[foundation_nutrient_df.name.isin(tracking_nutrition)].rename(columns={'id': 'nutrient_id'})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            477 non-null    int64  
 1   name          477 non-null    object 
 2   unit_name     477 non-null    object 
 3   nutrient_nbr  465 non-null    float64
 4   rank          466 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 18.8+ KB


In [133]:
foundation_result_df = foundation_food_nutrient_df.merge(foundation_nutrient_df, on="nutrient_id", how="left")
foundation_result_df = foundation_result_df[foundation_result_df.name.notna()]
foundation_result_df = foundation_result_df.groupby(['fdc_id', 'name'])['amount'].sum().unstack().reset_index("fdc_id")

In [134]:
foundation_result_df = foundation_result_df.merge(foundation_food_df, on="fdc_id", how="inner")
foundation_result_df = foundation_result_df[["fdc_id", "description", "Carbohydrate, by difference", "Protein", "Total lipid (fat)"]]

In [137]:
foundation_result_df = foundation_result_df.dropna(how='any')

In [145]:
foundation_result_df = foundation_result_df.groupby("description")[['Carbohydrate, by difference', "Protein", "Total lipid (fat)"]].mean().reset_index()

In [148]:
foundation_result_df

,description,"Carbohydrate, by difference",Protein,Total lipid (fat)
0,"Almond butter, creamy",21.236660,20.787340,53.04000
1,"Almond milk, unsweetened, plain, refrigerated",0.670750,0.656250,1.55800
2,"Almond milk, unsweetened, plain, shelf stable",0.337112,0.554688,1.22100
3,"Apple juice, with added vitamin C, from concen...",11.363962,0.085938,0.28630
4,"Apples, fuji, with skin, raw",15.675581,0.149219,0.16125
...,...,...,...,...
236,"Yogurt, Greek, plain, nonfat",3.640000,10.300000,0.37000
237,"Yogurt, Greek, plain, whole milk",4.754020,8.778880,4.39400
238,"Yogurt, Greek, strawberry, nonfat",12.200000,8.060000,0.15000
239,"Yogurt, plain, nonfat",8.076750,4.226750,0.08750
